#  Session 2 - The Forward Model

<div class="alert alert-success">
 
**By the end of this session you will be able to**

- Compute the moments of the output from a DMA operated at a single voltage.
- Compute the convolution matrix for DMAs with corresponding to a custom instrument.
- Apply the convolution matrix to compute the response function from a size distribution.

</div>

<div class="alert alert-danger">

**Instructions**
    
- Go to Menu "Cell" -> "Run All Below"
- The notebook will initialize all cells 
- Navigate back to this point and work through the notebook sequentially.
- When finished, move to Session 3.
 
</div>


## 1. Transmission through the DMA at Constant Voltage

In [ ]:
include("Scripts/startup.jl")
play("Audio/dma-selection1.ogg")

The Figure shows a schematic instrument setup with an array of detectors. The DMA is operated in classifier mode and set to a single voltage. The mobility classified particles are then passed to one or more detectors. 
<img src="Figures/nbs2_f01.png" width="350" align = "middle">
<figcaption><b> Figure. </b> Illustration of the combined transfer function through the DMA. </figcaption> 


<div class="alert alert-warning">
The net transmission function T describes transmission through the DMA, including the charge filter, the DMA transfer function, and the DMA transmission efficiency. 

```julia
T(zˢ,k,Λ,δ) = δ.Ω(Λ,δ.Z,zˢ/k).*δ.Tc(k,δ.Dp).*δ.Tl(Λ,δ.Dp)
```

where Λ is the DMA configuration, δ the DMA grid, zˢ is the z-star selected by the DMA, and k is the number of charges. 	
</div>


In [ ]:
play("Audio/dma-selection2.ogg")

In [ ]:
# The DMA fo this session
qsa,qsh = 1.66e-5, 8.33e-5                       # Qsample [m3 s-1], Qsheath [m3 s-1]
t,p = 295.15, 1e5                                # Temperature [K], Pressure [Pa]
r₁,r₂,l = 9.37e-3,1.961e-2,0.44369               # DMA geometry [m]
Λ = DMAconfig(t,p,qsa,qsh,r₁,r₂,l,0.0,:-,6,:cylindrical)  

# We need a low resolution and a high resultion grid for visualization purposes (this is slow!))
z₁,z₂ = vtoz(Λ,10000), vtoz(Λ,10)    # bins, upper, lower mobility limit
δ  = setupDMA(Λ, z₁, z₂, 60); 
δ1 = setupDMA(Λ, z₁, z₂, 512); 

In [ ]:
play("Audio/dma-selection3.ogg")

In [ ]:
# Define the mobility
zˢ = dtoz(Λ, 100*1e-9)     

# Define the DMA transmission function
T(zˢ,k,Λ,δ) = δ.Ω(Λ,δ.Z,zˢ/k).*δ.Tc(k,δ.Dp).*δ.Tl(Λ,δ.Dp)

# Plot the result
plot(x = δ.Dp, y = T(zˢ,1,Λ,δ), Geom.step, Guide.xlabel("Apparent + 1 Mobility Diameter (nm)"), Guide.ylabel("Fraction (-)"))

**Figure.** The shape is the transfer function, modulated by losses and charging efficiency and represented at the bin resolution of the DMA grid.

The transmission function ```T(zˢ,k,Λ,δ)``` represents a vector along the DMA grid that predicts the fraction of particles carrying k charges that transmit if the DMA voltage is set to selected zˢ. It is therefore a vector that is defined along the DMA mobility/diameter grid. For our DMA defined above we had 60 bins.

In [ ]:
play("Audio/dma-selection4.ogg")

<div class="alert alert-warning">
    
- zˢ is the z-star selected by the DMA
- 𝕟ᶜⁿ is an assumed known bimodal lognormal distribution computed on the DMA size grid
- ℕ is an array of transmitted mobility distributions carrying k charges
- 𝕄 is an array of transmitted apparent mobility distributions carrying k charges
- 𝕟ₜ, 𝕞ₜ are the superposition of these distributions
</div>

In [ ]:
T(zˢ,k,Λ,δ) = δ.Ω(Λ,δ.Z,zˢ/k).*δ.Tc(k,δ.Dp).*δ.Tl(Λ,δ.Dp)                   # Transmission model
zˢ = dtoz(Λ, 100*1e-9)                                                      # Mobility for 100 nm paticles
𝕟ᶜⁿ = DMALognormalDistribution([[900., 40., 1.5], [500., 180., 1.4]], δ1);  # Returns the size distribution 
ℕ = map(k -> T(zˢ,k,Λ,δ1)*𝕟ᶜⁿ,1:3)                                  # ℕ[k] : Mob. size distribution +k charges
𝕄 = map(k -> (ztod(Λ,1,zˢ)/ztod(Λ,k,zˢ))⋅(T(zˢ,k,Λ,δ1)*𝕟ᶜⁿ),1:3)   # 𝕄[k] : App. Mob. distribution +k charges 
𝕟ₜ, 𝕞ₜ = sum(ℕ), sum(𝕄)

include("Scripts/plot_transmission.jl")

<b> Figure. </b> Left: assumed bimodal lognormal size distribution. Middle: monodisperse mobility size distribution plotted against the apparent +1 mobility diameter, defined as the apparent setpoint diameter of the DMA. Dashed line is total number concentration. Right: same as middle panel but plotted
versus the mobility diameter.


The DMA selects approximately a triangular distribution around mobility centroid $z^s$. The distribution is symmetric when plotted against the log of mobility, but asymmetric when plotted against the log of the apparent +1 mobility diameter because the Cunningham slip flow correction factor applied in the conversion from mobility to diameter is a strong function of particle size. The majority of selected particles are singly charged, but the contribution of multiply charged particles to the total number is not negligible. The mobility distribution has contributions from particles that are at least twice the diameter of the selected centroid diameter. The relative fractions are determined by the equilibrium charge fraction and the number of particles available at each diameter.


<div class="alert alert-danger">
    
The ```map(k->f, 1:3)``` construct sequentially applies values from the array [1,2,3] to k and calls the function f. The output is an array of length 3. Since ```T(zˢ,k,Λ,δ)``` produces a vector, 𝕟ᶜⁿ is a size distribution, and vector * size distribution is a size distribution, the output of
    
```
ℕ = map(k -> T(zˢ,k,Λ,δ)*𝕟ᶜⁿ,1:3) 
```
    
is an array of size distributions.
</div>

<div class="alert alert-info">

1. Predict how the number concentration of 𝕄[3] and ℕ[3] will change if the number concentration of the accumulation mode were to decrease, i.e. change from 500 cm-3 to 5 cm-3.

2. Change the number concentration in the 
    ```𝕟ᶜⁿ = DMALognormalDistribution([[900., 40., 1.5], [500., 180., 1.4]], δ)```
    statement in the cell above and rerun the cell with CTRL-Enter. Compare with your prediction.

</div>

## 1.2 Example Application: Calibrate a Particle into Liquid Sampler


In [ ]:
play("Audio/dma-selection5.ogg")

<img src="Figures/pils.png" width="300" align = "left">

**Figure.** Example DMA-CPC-PILS setup for calibration.

In [ ]:
Number(ℕ) = sum(map(𝕟 -> sum(𝕟.N), ℕ))
Ncpc1 = sum(𝕟ₜ.N)
Ncpc2 = Number(ℕ)
println(Ncpc1, "   ", Ncpc2)  # Show that the two methods produce the same result are the same

### Naive Mass Estimate

The DMA is set to select a single apparent +1 mobility size, e.g. 100 nm in the example above. If we ignore the contribution of multiply charged particles, the mass selected is 

$M=\frac{\pi}{6}D_{p}^{3}N\rho_{p}$

where $\rho_{p}$ is the particle density. 

<div class="alert alert-warning">
Note that for mass calculations, if the density is $1000\; kg\;m^{-3} = 1\; g\; cm^{-3}$, the volume concentration in $\mu m^{3}\;cm^{-3}$ equals to the mass concentration in $\mu g\;m^{-3}$. This is an important conversion and explains why $\mu m^{3}\;cm^{-3}$ is the preferred unit for volume concentration.
</div>

In [ ]:
NaiveMass(Dp, ρp) = pi/6*Dp^3.0*sum(𝕟ₜ.N)  # Diameter in micron, and density in g cm-3
NaiveMass(0.1, 1.77)   

### Mass Accounting for Multiply Charged Particles

To compute the correct mass we must integrate over the distributions ℕ[1], ℕ[2], ℕ[3] and so forth, as we did with number, and weigh them by Dp^3. Because of the Dp^3 weighing the contribution of +2 and +3 charged particles to selected mass can be substantial. Depending on the shape of the size distribution

In [ ]:
# Diameter in micron, and density in g cm-3
Mass(ℕ, ρp) = sum(map(𝕟 -> sum(pi ./ 6 .* (𝕟.Dp .* 1e-3).^3.0 .* 𝕟.N .* ρp), ℕ))

println("Naive mass concentration   ", NaiveMass(0.1, 1.77))
println("Correct mass concentration  ", Mass(ℕ, 1.77))

<div class="alert alert-info">

1. Predict how the ratio of naive mass concentration to correct mass concentration will change if the number concentration of the accumulation mode were to decrease, i.e. change from 500 cm-3 to 5 cm-3. (Same exercise as previously.)

2. Change the number concentration in the 
    ```𝕟ᶜⁿ = DMALognormalDistribution([[900., 40., 1.5], [500., 180., 1.4]], δ)```
statement in the cell above and rerun all of the cells with CTRL-Enter. Compare with your prediction.

</div>
    
<div class="alert alert-success">

**You have met the following learning objective:**
    
- Compute the moments of output from a DMA operated at a single voltage.
    
</div>

## 2. The Fredholm Integral Equation

In [ ]:
play("Audio/fredholm1.ogg")

As worked out above, the net transmission function T describes transmission through the DMA, including the charge filter, the DMA transfer function, and the DMA transmission efficiency. 

```julia
T(zˢ,k,Λ,δ) = δ.Ω(Λ,δ.Z,zˢ/k).*δ.Tc(k,δ.Dp).*δ.Tl(Λ,δ.Dp)
```

Defining a zˢ and evaluating for all charges and integrating over size yields the total number concentration measured by the CPC. 

```julia
zˢ = dtoz(Λ, 100*1e-9) 
ℕ = map(k -> T(zˢ,k,Λ,δ1)*𝕟ᶜⁿ,1:k)
𝕟ₜ = sum(ℕ)
Ncpc = sum(𝕟ₜ.N)
```

In [ ]:
play("Audio/fredholm2.ogg")

Formally, the response in channel i (corresponding to a transmission through the DMA at a single voltage) is given by the Fredholm convolution integral 

 <center> $ R_i = \int_{z_a}^{z_b} \sum_{k=1}^m \Omega(z,Z_{i,k}^s)T_c(D[z,1])T_l(D[z,1])\frac{dN}{d\ln D}\frac{d \ln D}{dz}dz \;\;\;\;\;\;\ i = 1,2...,n$   </center>
 
The integral is performed over the limits $z_a$ and $z_b$, which corresponds to the upper and lower mobility limit set by the voltage range used to operate the DMA. The function $\frac{dN}{d\ln D}\frac{d\ln D}{dz}dz$ evaluates to the number concentration of particles that lie in the interval $[z,z + dz]$. Note that $D[z,1]$ is used in the charge filter and loss function since the integral is applied over the transform of the selected centroid mobility $Z_{i,k}^s$ (see above). $Z$ is the mobility vector of the grid and the subscript $i$ denotes the response channel. 

In [ ]:
play("Audio/fredholm3.ogg")

The convolution integral can be discretized:  <br><br>

 <center> $ R_i = \sum_{j=1}^n \left [ \sum_{k=1}^m \Omega(Z_j,Z_{i,k}^s)T_c(D[Z_j,1])T_l(D[Z_j,1])N(Z_j) \right] $</center> <br>

$N(Z_j)$ is the the number concentration of particles in the $j^{th}$ bin, $i = 1...n$
are indices the observed instrument channel, $j = 1...n$ are indices of the physical size bins, and $k = 1...m$ are indices of charges carried by the particle. Here it is assumed that $\Omega(Z_j,Z_{i,k}^s)$ can be approximated being constant over the bin $[Ze_{j},Ze_{j+1}]$, which is only true if a sufficiently large number of size bins is used. The discretized version represents a set of $n$ equations that can be written in matrix form <br><br>

<center> 
$ \begin{bmatrix}
       R_1    \\
       \vdots \\        
       R_n
\end{bmatrix} =
\begin{pmatrix}
  \sum_{k=1}^m \Omega(Z_1,Z_{1,k}^s)T_c(D[Z_1,1])T_l(D[Z_1,1])  & \cdots & \sum_{k=1}^m \Omega(Z_n,Z_{1,k}^s)T_c(D[Z_n,1])T_l(D[Z_n,1]) \\
  \vdots  & \ddots  & \vdots  \\
  \sum_{k=1}^m \Omega(Z_1,Z_{n,k}^s)T_c(D[Z_1,1])T_l(D[Z_1,1])  & \cdots & \sum_{k=1}^m \Omega(Z_n,Z_{n,k}^s)T_c(D[Z_n,1])T_l(D[Z_n,1])  
\end{pmatrix}
\begin{bmatrix}
       N_1    \\[0.3em]
       \vdots \\[0.3em]        
       N_n
\end{bmatrix}
$</center> <br><br>



<div class="alert alert-warning">

The discrete version of the Fredholm equation can thus be cast as the following
    
   <center> $R = \mathbf{A}N$ </center>

where $R$ is the response vector, $\mathbf{A}$ is the convolution matrix, and $N$ is the discretized true number concentration.
</div>



<div class="alert alert-danger">
    
    
Note how the terms comprising the convolution matrix 
    
   <center> $ \Omega(Z_1,Z_{1,k}^s)T_c(D[Z_1,1])T_l(D[Z_1,1]) $ </center>
    
are the transmission function defined in code
    
```julia
T(zˢ,k,Λ,δ) = δ.Ω(Λ,δ.Z,zˢ/k).*δ.Tc(k,δ.Dp).*δ.Tl(Λ,δ.Dp)
```
</div>

In [ ]:
play("Audio/fredholm4.ogg")

<div class="alert alert-warning">

**Generic solution to solve for the convolution matrix**
    
The convolution matrix is of size $n\times n$, where $n$ is the number of bins. It can be computed via the following expression (Petters, 2018):

```julia
T(zˢ,k,Λ,δ) = δ.Ω(Λ,δ.Z,zˢ/k).*δ.Tc(k,δ.Dp).*δ.Tl(Λ,δ.Dp)
𝐀 = (hcat(map(zˢ->Σ(k->T(zˢ,k,Λ,δ),Λ.m),δ.Z)...))'
```
</div>


In [ ]:
T(zˢ,k,Λ,δ) = δ.Ω(Λ,δ.Z,zˢ/k).*δ.Tc(k,δ.Dp).*δ.Tl(Λ,δ.Dp)
𝐀 = (hcat(map(zˢ->Σ(k->T(zˢ,k,Λ,δ),Λ.m),δ.Z)...))'

<div class="alert alert-info">

Write an expression that computes the convolution matrix for your DMA, defined as myΛ and myδ.

</div>

Note that you can not only define your own instrument configuration, but also change the convolution terms. For example, you can omit the charge filter, you can omit the transmission function through the DMA. In practice, these are just short computer functions and you can replace them with your own as needed. For example you swap out the transfer function to a custom version, or add a function that describes the detector efficiency.

<div class="alert alert-info">

Write and expression that computes the convolution matrix for your DMA, defined as myΛ and myδ, but without accounting for transmission efficiency through the DMA.

</div>

<div class="alert alert-success">

**You have met the following learning objective:**
    
- Compute the convolution matrix for DMAs with corresponding to a custom instrument.
    
</div>

In [ ]:
play("Audio/fredholm5.ogg")

The response function can be computed from the convolution matrix and the known true size distribution

<img src="Figures/nbs2_f02.png" width="750" align = "middle">
<b> Figure. </b> Convolution of true size distribution resulting in the response vector. <br> <br>

<div class="alert alert-warning">

In the section on size distribution arithmetic I included the operation matrix * size distribution. Thus we can conveniently write 

𝕣 = 𝐀 * 𝕟

to compute the response function
    
</div>


In [ ]:
# Here is a simple example
𝕟 = DMALognormalDistribution([[400, 30, 1.2],[500, 110, 1.7]], δ)
𝕣 = 𝐀 * 𝕟

include("Scripts/plot_response_function.jl")

<div class="alert alert-success">

**You have met the following learning objective:**
    
- Apply the convolution matrix to compute the response function from a size distribution.
    
</div>
